In [ ]:
!pip install larq

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=f808b5ef84fd2f19c9d738fc68cbc84eb0e1aca250333adfcb163d87107eeed5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [ ]:
import tensorflow as tf
import larq as lq

import keras.backend as K
from keras.datasets import mnist
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler

In [ ]:
batch_size = 100
epochs = 20

# network
num_units = 4096
hidden_layers = 3
use_bias = False

# learning rate schedule
lr_start = 1e-3
lr_end = 1e-4
lr_decay = (lr_end / lr_start)**(1. / epochs)

# BN
e = 1e-6
momentum = 0.9

# dropout
drop_hidden = 0.5

In [17]:
# Import MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten images
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

# Normalize pixel values 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, 10) * 2 - 1 # -1 or 1 for hinge loss
Y_test = np_utils.to_categorical(y_test, 10) * 2 - 1

60000 train samples
10000 test samples


In [18]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              kernel_initializer='glorot_uniform')

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantDense(num_units, 
                               kernel_quantizer="ste_sign",
                               kernel_constraint="weight_clip",
                               use_bias=use_bias,
                               input_shape=(784,), 
                               name='dense{}'.format(1)))
model.add(tf.keras.layers.BatchNormalization(epsilon=e, momentum=momentum, name='bn{}'.format(1)))
model.add(tf.keras.layers.Dropout(drop_hidden, name='drop{}'.format(1)))

for i in range(1, hidden_layers):
  model.add(lq.layers.QuantDense(num_units, use_bias=True, name='dense{}'.format(i+1), **kwargs))
  model.add(tf.keras.layers.BatchNormalization(epsilon=e, momentum=momentum, name='bn{}'.format(i+1)))
  model.add(tf.keras.layers.Dropout(drop_hidden, name='drop{}'.format(i+1)))

# L2 SVM Output layer
model.add(tf.keras.layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2()))
model.add(tf.keras.layers.Activation('linear'))


lq.models.summary(model)

+sequential_7 stats---------------------------------------------------------------------------+
| Layer         Input prec.     Outputs   # 1-bit  # 32-bit   Memory  1-bit MACs  32-bit MACs |
|                     (bit)                   x 1       x 1     (kB)                          |
+---------------------------------------------------------------------------------------------+
| dense1                  -  (-1, 4096)   3211264         0   392.00           0      3211264 |
| bn1                     -  (-1, 4096)         0      8192    32.00           0            0 |
| drop1                   -  (-1, 4096)         0         0        0           ?            ? |
| dense2                  1  (-1, 4096)  16777216      4096  2064.00    16777216            0 |
| bn2                     -  (-1, 4096)         0      8192    32.00           0            0 |
| drop2                   -  (-1, 4096)         0         0        0           ?            ? |
| dense3                  1  (-1, 4096) 

In [19]:
model.compile(tf.keras.optimizers.Adam(lr=0.01, decay=0.0001),
              loss='squared_hinge',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs)

test_loss, test_acc = model.evaluate(X_test, Y_test)

print(f"Test accuracy {test_acc * 100:.2f} %")

Epoch 1/20
600/600 [==============================] - 524s 873ms/step - loss: 1.3835 - accuracy: 0.8050
Epoch 2/20
600/600 [==============================] - 546s 909ms/step - loss: 0.3315 - accuracy: 0.8883
Epoch 3/20
600/600 [==============================] - 524s 873ms/step - loss: 0.1025 - accuracy: 0.9351
Epoch 4/20
600/600 [==============================] - 529s 882ms/step - loss: 0.1060 - accuracy: 0.9417
Epoch 5/20
600/600 [==============================] - 529s 882ms/step - loss: 0.1118 - accuracy: 0.9414
Epoch 6/20
600/600 [==============================] - 511s 851ms/step - loss: 0.0735 - accuracy: 0.9531
Epoch 7/20
600/600 [==============================] - 513s 855ms/step - loss: 0.0653 - accuracy: 0.9595
Epoch 8/20
600/600 [==============================] - 511s 852ms/step - loss: 0.0591 - accuracy: 0.9626
Epoch 9/20
600/600 [==============================] - 512s 853ms/step - loss: 0.0540 - accuracy: 0.9654
Epoch 10/20
600/600 [==============================] - 515s 859m